# Подготовка данных

### Теперь приготовим данные для поиска. Далее будем искать их с помощью библиотек, умеющих анализировать человеческий язык

In [39]:
from bs4 import BeautifulSoup
# функция walk дает рекурсивное получение имен файлов в дереве каталогов
from os import walk

# модуль difflib содержит классы и функции для сравнения последовательностей (текстов)
import difflib
import re
# модуль символьной перекодировки 
import codecs
# from os import makedirs

import pandas as pd
import numpy as np

# nltk - модуль для символьной и статистической обработки естественного языка
import nltk
import pymorphy2
from string import punctuation

In [40]:
pages_list = []

# собираем список страниц, которые есть, для этого: проходим по raw_pages:
for dirpath, dirnames, filenames in walk('data/raw_pages'):
    if '.ipynb_checkpoints' in dirpath:
        continue
    
    dirpath = dirpath.replace('\\', '/') # для Windows
    for fn in filenames:
        if '.DS_Store' in fn:
            continue 
            
        fp = f'{dirpath}/{fn}'
        pages_list.append(fp)

In [41]:
pages_list[:5]

['data/raw_pages/data+raw_pages+zrg74.ru+obshhestvo+item+26920-chistovoe-vyrazhenie-v-zlatouste-oglasili-sroki-sdachi-10-jetazhki-dlja-vethoavarijshhikov.html.txt',
 'data/raw_pages/data+raw_pages+zrg74.ru+obshhestvo+item+26924-verh-masterstva-v-zlatouste-blagoustrojstvo-jekotropy-urenga-zavershajut-rabotami-na-vysote.html.txt',
 'data/raw_pages/data+raw_pages+zrg74.ru+obshhestvo+item+26932-kanikuljarnyj-podschjot-bolee-6-tysjach-junyh-zlatoustovcev-otdohnuli-jetim-letom-v-lagerjah-i-zdravnicah.html.txt',
 'data/raw_pages/data+raw_pages+zrg74.ru+obshhestvo+item+26934-takoj-variant-zhiteli-zlatousta-mogut-projti-perepis-v-mfc.html.txt',
 'data/raw_pages/data+raw_pages+zrg74.ru+obshhestvo+item+26936-po-snezhnym-koridoram-zhitelej-zlatousta-posvjatili-v-plany-po-sozdaniju-novogodnih-ploshhadok.html.txt']

In [42]:
def remove_script(file):
    """Функция облегчает жизнь difflib'у и удаляет скрипты, футеры и хедеры"""
    # первый аргумент здесь - html_doc, второй - тип парсера 
    soup = BeautifulSoup(''.join(file), 'html.parser')
    for s in soup.select('script'):
        # extract()удаляет тег или строку из дерева 
        # и возвращает тег или строку, которые были извлечены
        s.extract()
        
    for f in soup.select('footer'):
        f.extract()
    
    # селектор CSS для проанализированного документа:  возвращает все совпадающие элементы;
    # soup.select("body a") - достанет тело данного тега;
    # soup.select("title") --> # [<title>The Dormouse's story</title>]
    for f in soup.select('header'):
        f.extract()
        
    return str(soup).split('\n')

In [43]:
# Теперь из полученного списка берем 2 первых файла и очищаем их функцией remove_script:
fp_1 = 'data/raw_pages/zrg74.ru/obshhestvo/item/26920-chistovoe-vyrazhenie-v-zlatouste-oglasili-sroki-sdachi-10-jetazhki-dlja-vethoavarijshhikov.html'
fp_2 = 'data/raw_pages/zrg74.ru/obshhestvo/item/26924-verh-masterstva-v-zlatouste-blagoustrojstvo-jekotropy-urenga-zavershajut-rabotami-na-vysote.html'

with codecs.open(fp_1, 'r', 'utf_8_sig') as f:
    file1 = remove_script(f.readlines())
with codecs.open(fp_2, 'r', 'utf_8_sig') as f:
    file2 = remove_script(f.readlines())

In [44]:
def clean_diff(diff): 
    """Функция очиcтки (пригодится позже)"""
    diff = re.sub('\<[^\<\>]+\>', ' ', diff)
    diff = re.sub('&nbsp;', ' ', diff)
    diff = re.sub('\xa0', ' ', diff)
    diff = re.sub('\s\s+', ' ', diff)
    diff = re.sub('^[\+\-] ', '', diff)
    diff = re.sub('[^\s]*(jeg|background|li|widget|a:hover|comment|button|page|color|div|split|cartdetail|current|ul|a|span|owl|entry|tr|height|border|content|quantity|jnews|sub|#|px|url|width|Roboto|h|btn|none|t|woocommerce|img|pricegroup|p|Мы в социальных сетях|Метки|тэги|О нас|Тэги|Златоустовский рабочий|Новости Златоуста)[^\s]*', '', diff)
    diff = re.sub('\>', '', diff)
    diff = re.sub('(\{  : ; \})|(\{  :  \})|(\{  : ;  :   : ;  : ; \})|(\{  : ;  : ;  : ; \})|(\{  :   : ; \})|(\{  : ;  : ; \})|(\(\{    \})|(\{  : ;  : ;  :   : ; \})|(\{  :   : ;  :   : ; \})|(\{  :   :  \})|(\{  : ;  :  \})|(\{  : %; \})|(\{  1; \})|(\{   \})|col-md-7', '', diff)
    diff = re.sub('(\s){2,50}', '', diff)
    
    return diff

In [45]:
# теперь сравним все страницы с «эталонной»: за нее возьмем первую же ссылку.
# difflib работает так, что сравнивает правый документ с «эталоном» слева.
# вновь появившийся текст (справа) функция пометит плюсом 
# (также спорные моменты – знак ?, пустой символ – без изменений, 
# минус – текст, который был слева, но отсутствует в документе справа). Нам нужны только плюсы 
page_lines = []
# ndiff в модуле in difflib позволяет получить различия между двумя текстами при их сравнении
for diff in difflib.ndiff(file1, file2): # с предобработкой данных
    if re.search('^\+ ', diff) is None:
        continue  
    diff = clean_diff(diff)
    if len(diff) == 0:
        continue
    page_lines.append(diff)
#     print(diff)
page_text = ' '.join(page_lines)

# print(page_text)


In [46]:
def extract_page_text(file, origin_file):
    """Функция объединяет методы, описанные выше в коде"""
    page_lines = []
    for diff in difflib.ndiff(origin_file, file):
        if re.search('^\+ ', diff) is None:
            continue

        diff = clean_diff(diff)

        if len(diff) == 0:
            continue

        page_lines.append(diff)

    page_text = '. '.join(page_lines)
    
    return page_text

In [47]:
# Извлекаем тексты и сохраняем в файлы


page_texts_path = 'data/raw_pages/'
create_fp = lambda fp: fp.replace('/', '+').replace('\\', '+') + '.txt'


with open(pages_list[0], 'r', encoding="utf-8") as f:
    origin_file = remove_script(f.readlines())

with open(pages_list[1], 'r', encoding="utf-8") as f:
    file = remove_script(f.readlines())

# сравниваем текст с "эталонным":
page_text = extract_page_text(origin_file, file)
# page_text
try:
    with open(page_texts_path + create_fp(pages_list[0]), 'w', encoding="utf-8") as f:
        f.write(page_text)
except FileNotFoundError:
        print(f"Запрашиваемый файл {page_texts_path + create_fp(pages_list[0])} не найден")

for fp in pages_list[1:]:
    with open(fp, 'r', encoding="utf-8") as f:
        file = remove_script(f.readlines())
    
    page_text = extract_page_text(file, origin_file)
    with open(page_texts_path + create_fp(fp), 'w', encoding="utf-8") as f:
        f.write(page_text)

In [65]:
pages_texts = {}

for dirpath, dirnames, filenames in walk(page_texts_path):
    if '.ipynb_checkpoints' in dirpath:
        continue
        
    for fn in filenames:
        if '.DS_Store' in fn:
            continue
            
        fp = f'{dirpath}/{fn}'
        with open(fp, 'r', encoding="utf-8") as f:
            pt = ' '.join(f.readlines())
        
        original_link = fp.split('/')[-1].replace('|', '/').replace('+', '/').replace('.txt', '')
        pages_texts[original_link] = pt

### Займемся работой непосредственно текстами

In [66]:
# pymorphy2 - морфологический анализатор для русского языка, написанный на языке Python и использующий словари из OpenCorpora
# !pip install pymorphy2
# Словари распространяются отдельным пакетом:
# !pip install pymorphy2-dicts

# ставим либо DAWG-Python, либо DAWG (он быстрее)
# Этот пакет pure-python обеспечивает доступ 
# только для чтения к файлам, созданным библиотекой dawgdic C ++ и пакетом DAWG python
# Основная цель DAWG-Python - предоставить доступ к DAWG, 
# не требуя скомпилированных расширений
# !pip install DAWG-Python

In [67]:
# выведем знаки пунктуации
print(punctuation)

# В pymorphy2 для морфологического анализа русских слов имеется класс MorphAnalyzer
MORPH = pymorphy2.MorphAnalyzer()

def preprocess_text(text):
    text = text.lower()
    # множество пробелов заменяется на один:
    text = re.sub('\s\s+', ' ', text)
    return text

def extract_sentences(text):
    text = preprocess_text(text)

    sentences = []
    # sent_tokenize() - токенизация на уровне предложений:
    for s in nltk.sent_tokenize(text):
        wrds = []
        # word_tokenize() - токенизация на уровне слов:
        for wrd in nltk.word_tokenize(s):
            if wrd in punctuation:
                continue
            # берем, по умолчанию, первый из доступных разборов слова MORPH.parse(wrd)[0] в его нормальной форме 
            wrd = MORPH.parse(wrd)[0].normal_form
            wrds.append(wrd)
            
        clear_sentence = ' '.join(wrds)
        
        sentences.append(clear_sentence)
    
    return sentences

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


Loading dictionaries from C:\Users\zlatt\anaconda3\lib\site-packages\pymorphy2_dicts_ru\data
format: 2.4, revision: 417127, updated: 2020-10-11T15:05:51.070345


In [68]:
# Сделаем проверку того, как работает анализатор
MORPH.parse('березой')[0]

Parse(word='берёзой', tag=OpencorporaTag('NOUN,inan,femn sing,ablt'), normal_form='берёза', score=1.0, methods_stack=((DictionaryAnalyzer(), 'берёзой', 55, 4),))

In [69]:
nltk.sent_tokenize('Винсент ван Гог писал много. Ценители счастливы, когда рассматривают его картины')

['Винсент ван Гог писал много.',
 'Ценители счастливы, когда рассматривают его картины']

In [70]:
nltk.word_tokenize('Винсент ван Гог писал много - 1700руб./картина')

['Винсент', 'ван', 'Гог', 'писал', 'много', '-', '1700руб./картина']

In [71]:
len(pages_texts)

20

In [72]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zlatt\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [87]:
full_link_df = pd.DataFrame(columns=['link', 'sentence_order', 'sentence_text'])

for link, text in pages_texts.items():
    sentences = extract_sentences(text) 
    sentence_count = len(sentences)
#     for sent in sentences:
#         for wr in sent:
#             wr = wr.decode('utf-8', 'replace')
    
    link_df = pd.DataFrame({'link': np.repeat(link, sentence_count),
                            'sentence_order': np.arange(sentence_count),
                            'sentence_text': sentences})
    
    full_link_df = full_link_df.append(link_df)

In [88]:
full_link_df

,link,sentence_order,sentence_text
0,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,0,--
1,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,1,чистовой выражение
2,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,2,в златоуст огласить срок сдача 10-этажка для в...
3,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,3,
4,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,4,1em
...,...,...,...
401,26959-rabota-ne-dlja-galochki-zlatoustovec-pov...,401,\u0421\u0447\u0438\u0442\u0430\u044e \u0447\u0...
402,26959-rabota-ne-dlja-galochki-zlatoustovec-pov...,402,\u042d\u0442\u0438 \u0434\u0430\u043d\u043d\u0...
403,26959-rabota-ne-dlja-galochki-zlatoustovec-pov...,403,\u041e\u043d\u0438 \u043f\u043e\u043c\u043e\u0...
404,26959-rabota-ne-dlja-galochki-zlatoustovec-pov...,404,\u041a\u0440\u043e\u043c\u0435 \u0442\u043e\u0...


In [89]:
full_link_df.to_csv('data/link_page_sentences.csv', index=False)

In [90]:
pd.read_csv('data/link_page_sentences.csv').head()

,link,sentence_order,sentence_text
0,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,0,--
1,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,1,чистовой выражение
2,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,2,в златоуст огласить срок сдача 10-этажка для в...
3,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,3,NaN
4,data/raw_pages/zrg74.ru/obshhestvo/item/26920-...,4,1em


In [91]:
# !pip install pyarrow
# Эта библиотека предоставляет Python API для функций, 
# предоставляемых библиотеками Arrow C ++, 
# а также инструменты для интеграции Arrow 
# и взаимодействия с pandas, NumPy и 
# другим программным обеспечением в экосистеме Python

You should consider upgrading via the 'c:\users\zlatt\anaconda3\python.exe -m pip install --upgrade pip' command.
